# Check the shapefile

Looking for null geometries, invalid geometries, and overlaps in the shapefile.

In [2]:
import geopandas as gpd
import s3fs
from shapely.validation import explain_validity
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import folium

In [3]:
# set up S3 filesystem
s3 = s3fs.S3FileSystem(anon=True) 

### Read in shapefile

In [4]:
# path to shapefile on S3
s3_shapefile_path = "s3://dea-public-data-dev/projects/WIT/BWS_MDBA_ANAE_WIT_05_2022/shp_file/ANAEv3_WIT_clean19042022/ANAEv3_WIT_clean.shp"

# load
shapefile = gpd.read_file(s3_shapefile_path, engine='fiona')

print(f"Shapefile loaded. Number of rows: {len(shapefile)}")

Shapefile loaded. Number of rows: 270653


In [5]:
shapefile

,UID,ANAE_TYPE,Area_Ha,geometry
0,r1dtvqu5m,F4: Unspecified riparian zone or floodplain,3.972622,"POLYGON ((611266.353 -3895503.720, 611256.319 ..."
1,r1dtygfug,Pt2.1.2: Temporary tall emergent marsh,3.316279,"POLYGON ((617367.374 -3897620.931, 617370.374 ..."
2,r1dtyggsf,Etd1.2.1: Tide dominated saltmarsh,2.217581,"POLYGON ((617038.183 -3897457.937, 617042.095 ..."
3,r1dtyjqxg,Pt4.2: Temporary wetland,5.736414,"POLYGON ((614794.869 -3896524.926, 614794.833 ..."
4,r1dtyjxp1,Pt2.1.2: Temporary tall emergent marsh,4.490608,"POLYGON ((614794.869 -3896524.926, 614784.178 ..."
...,...,...,...,...
270648,r7hh7kxjv,Rt1.1: Temporary high energy upland stream,1.008318,"POLYGON ((1949229.310 -3119406.198, 1949232.82..."
270649,r7hh7m4fc,Rt1.1: Temporary high energy upland stream,2.508319,"POLYGON ((1948684.816 -3119059.552, 1948649.94..."
270650,r7hhdb24d,Rt1.1: Temporary high energy upland stream,3.858787,"POLYGON ((1946809.814 -3116801.769, 1946769.42..."
270651,r7hhdvgdj,Rt1.1: Temporary high energy upland stream,1.213341,"POLYGON ((1947536.645 -3113549.088, 1947530.54..."


In [6]:
shapefile.crs

<Projected CRS: EPSG:3577>
Name: GDA94 / Australian Albers
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Australia - Australian Capital Territory; New South Wales; Northern Territory; Queensland; South Australia; Tasmania; Western Australia; Victoria.
- bounds: (112.85, -43.7, 153.69, -9.86)
Coordinate Operation:
- name: Australian Albers
- method: Albers Equal Area
Datum: Geocentric Datum of Australia 1994
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [7]:
# check UID is unique
unique_id_column_name = "UID"
len(shapefile[unique_id_column_name]) == len(set(shapefile[unique_id_column_name]))

True

### Check for null geometries e.g. rows where the geometry is null

In [8]:
null_geoms = shapefile[shapefile.geometry.isnull()]
if not null_geoms.empty:
    print(f"❗ Rows with null geometries: {null_geoms.index.tolist()}")
else:
    print("✅ No null geometries found.")

✅ No null geometries found.


In [9]:
# remove any null geometries 
shapefile_cleaned = shapefile[shapefile.geometry.notnull()]
print(f"Removed {len(shapefile) - len(shapefile_cleaned)} rows with null geometries.")

Removed 0 rows with null geometries.


### Check for invalid geometries e.g. self intersecting geometries

In [10]:
invalid_geoms = shapefile_cleaned[~shapefile_cleaned.is_valid]

if not invalid_geoms.empty:
    print(f"❗ Found {len(invalid_geoms)} invalid geometries.")
    
    for idx, row in invalid_geoms.iterrows():
        uid = row["UID"]  
        reason = explain_validity(row.geometry)
        print(f" - UID {uid} is invalid: {reason}")
else:
    print("✅ All remaining geometries are valid.")

❗ Found 103 invalid geometries.
 - UID r1dvdkh4n is invalid: Ring Self-intersection[616620.8884 -3897348.5196]
 - UID r1dvvs1z3 is invalid: Ring Self-intersection[642844.7095 -3899094.5659]
 - UID r1ehbstnr is invalid: Ring Self-intersection[654835.918900002 -3918711.1496]
 - UID r1guqts4s is invalid: Ring Self-intersection[783911.107500002 -3779124.8943]
 - UID r1tfzwdqx is invalid: Ring Self-intersection[1024539.3454 -3992301.2575]
 - UID r1uj0742d is invalid: Ring Self-intersection[791321.572299998 -3766319.6445]
 - UID r1un4pgvs is invalid: Ring Self-intersection[802685.8171 -3744969.6027]
 - UID r1urmbm7x is invalid: Ring Self-intersection[849084.671399999 -3728107.593]
 - UID r1utcekuh is invalid: Ring Self-intersection[862204.1538 -3756319.8586]
 - UID r1v745cjf is invalid: Ring Self-intersection[956435.158799998 -3816876.208]
 - UID r1ve34q75 is invalid: Ring Self-intersection[984773.8068 -3816785.8301]
 - UID r1w74kp6m is invalid: Ring Self-intersection[1072528.2993 -3983667.9

In [11]:
# remove invalid geometries
shapefile_cleaned = shapefile_cleaned[shapefile_cleaned.is_valid]
print(f"Remaining rows after cleaning: {len(shapefile_cleaned)}")

Remaining rows after cleaning: 270550


### Check for intersecting polygons 
We remove null or invalid geometries because testing for intersections could result in weird results.

This code
1. creates a spatial index (e.g. spatial look up table) based on the bounding boxes of the geometries. this means you can quickly filter out geometries that are near another without checking every one manually.
2. it then finds possible overlaps using the bounding boxes, which is much faster than checking the actual shapes
3. it then checks for real overlaps using intersect, touches is used to exclude features that only touch at a point or line which is not technically an overlap
4. records the UIDs of overlapping pairs

In [14]:
# build spatial index for fast lookup
sindex = shapefile_cleaned.sindex

overlap_pairs = set()

# loop through each geometry and compare it to potential overlaps
for idx, geom in shapefile_cleaned.geometry.items():
    if geom is None or geom.is_empty:
        continue  # skip if geometry is missing or empty

    # get possible matches using bounding boxes
    possible_matches_index = list(sindex.intersection(geom.bounds))
    
    # filter to actual overlaps (excluding itself)
    for match_idx in possible_matches_index:
        if idx >= match_idx:
            continue  # avoid duplicates and self

        other_geom = shapefile_cleaned.geometry.iloc[match_idx]
        if other_geom is None or other_geom.is_empty:
            continue

        if geom.intersects(other_geom) and not geom.touches(other_geom):
            # record a sorted tuple to prevent duplicatepairings (e.g., (a, b) vs (b, a))
            uid1 = shapefile_cleaned.loc[idx, "UID"]
            uid2 = shapefile_cleaned.iloc[match_idx]["UID"]
            overlap_pairs.add(tuple(sorted((uid1, uid2))))

#  results
if overlap_pairs:
    print(f"❗ Found {len(overlap_pairs)} overlapping UID pairs.")
    for pair in sorted(overlap_pairs):
        print(f" - {pair[0]} overlaps with {pair[1]}")
else:
    print("✅ No overlapping geometries found.")

❗ Found 44 overlapping UID pairs.
 - r1dvcwur6 overlaps with r1dyq4xpv
 - r1dwnvrtu overlaps with r1dwpvmtd
 - r1dwpvmtd overlaps with r1dy0jxgk
 - r1dy632dn overlaps with r1dyq4xpv
 - r1dy6qbkv overlaps with r1dyq4xpv
 - r1epf61zy overlaps with r1epsb3cp
 - r1g4r71z2 overlaps with r1g4rk8t7
 - r1g4r752g overlaps with r1g4rk8t7
 - r1g4rk8t7 overlaps with r1g4rxk1d
 - r1g4xbxzg overlaps with r1g6811c0
 - r1g727wce overlaps with r1g72sjn5
 - r1g72u997 overlaps with r1g72y2sw
 - r1g789b0p overlaps with r1g78chp1
 - r1gm39sc6 overlaps with r1gm3egu8
 - r1gmd52n0 overlaps with r1gmf6x3e
 - r1gmf6x3e overlaps with r1gmfknh5
 - r1gmf6x3e overlaps with r1gmfqk8y
 - r1gtbsgtq overlaps with r1gw08kde
 - r1gtbstzm overlaps with r1gtbveyx
 - r1gtbtvze overlaps with r1gw08kde
 - r1gtbtw7g overlaps with r1gw08kde
 - r1gtbty9h overlaps with r1gw08kde
 - r1gtbwyy1 overlaps with r1gw08kde
 - r1gtbxjr6 overlaps with r1gw08kde
 - r1gtbxur1 overlaps with r1gw08kde
 - r1gtbz31s overlaps with r1gw08kde
 - r

Running the code on the non-cleaned shapefile to see what happens

In [15]:
# build spatial index for fast lookup
sindex = shapefile.sindex

overlap_pairs = set()

# loop through each geometry and compare it to potential overlaps
for idx, geom in shapefile.geometry.items():
    if geom is None or geom.is_empty:
        continue  # skip if geometry is missing or empty

    # get possible matches using bounding boxes
    possible_matches_index = list(sindex.intersection(geom.bounds))
    
    # filter to actual overlaps (excluding itself)
    for match_idx in possible_matches_index:
        if idx >= match_idx:
            continue  # avoid duplicates and self

        other_geom = shapefile.geometry.iloc[match_idx]
        if other_geom is None or other_geom.is_empty:
            continue

        if geom.intersects(other_geom) and not geom.touches(other_geom):
            # record a sorted tuple to prevent duplicate pairings (e.g., (a, b) vs (b, a))
            uid1 = shapefile.loc[idx, "UID"]
            uid2 = shapefile.iloc[match_idx]["UID"]
            overlap_pairs.add(tuple(sorted((uid1, uid2))))

#  results
if overlap_pairs:
    print(f"❗ Found {len(overlap_pairs)} overlapping UID pairs.")
    for pair in sorted(overlap_pairs):
        print(f" - {pair[0]} overlaps with {pair[1]}")
else:
    print("✅ No overlapping geometries found.")

❗ Found 61 overlapping UID pairs.
 - r1dvcwur6 overlaps with r1dyq4xpv
 - r1dwnvrtu overlaps with r1dwpvmtd
 - r1dwpvmtd overlaps with r1dy0jxgk
 - r1dy632dn overlaps with r1dyq4xpv
 - r1dy6qbkv overlaps with r1dyq4xpv
 - r1epf61zy overlaps with r1epsb3cp
 - r1g01tjpb overlaps with r1g01ud2n
 - r1g4r71z2 overlaps with r1g4rk8t7
 - r1g4r752g overlaps with r1g4rk8t7
 - r1g4rk8t7 overlaps with r1g4rkfwk
 - r1g4rk8t7 overlaps with r1g4rmfx9
 - r1g4rk8t7 overlaps with r1g4rxk1d
 - r1g4xbxzg overlaps with r1g6811c0
 - r1g727wce overlaps with r1g72sjn5
 - r1g72u997 overlaps with r1g72y2sw
 - r1g789b0p overlaps with r1g78chp1
 - r1gm39sc6 overlaps with r1gm3egu8
 - r1gmd52n0 overlaps with r1gmf6x3e
 - r1gmf6x3e overlaps with r1gmfknh5
 - r1gmf6x3e overlaps with r1gmfqk8y
 - r1gtbsgtq overlaps with r1gw08kde
 - r1gtbstzm overlaps with r1gtbveyx
 - r1gtbtvze overlaps with r1gw08kde
 - r1gtbtw7g overlaps with r1gw08kde
 - r1gtbty9h overlaps with r1gw08kde
 - r1gtbwyy1 overlaps with r1gw08kde
 - r